In [12]:
# written on 3.27.24 by @caramnix 

# resources: 
# https://platform.openai.com/docs/api-reference/chat/create
# https://platform.openai.com/docs/guides/text-generation/chat-completions-api
# https://github.com/openai/openai-python


In [49]:
import pandas as pd
import openai
import time
import requests
import os
from openai import OpenAI

In [50]:
# read in full dataset  
df= pd.read_csv("/Users/caranix/Library/CloudStorage/OneDrive-TheOhioStateUniversity/CS MS/CSPAN/115th_transcript_data/cleaned_transcript_data_115th copy.csv")

# get rid of duplicates 
df2= df.drop_duplicates(subset=['cleaned_text'])

In [51]:
def get_party(data):
    if isinstance(data, (float)): 
        return "NULL"
    else:
        return(data[0])

df2["party"] = df2["party_state"].str.split("-").apply(get_party)

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_33155/245730034.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["party"] = df2["party_state"].str.split("-").apply(get_party)


In [52]:
df2 = df2[df2.party != "NULL"]

In [53]:
len(df2)

2632

In [47]:
sum(df2.party =="D")

1269

In [48]:
sum(df2.party =="R")

1363

In [54]:
# Initialize the OpenAI API with your API key

client = OpenAI(
    api_key= "YOUR API KEY HERE",
)


In [55]:
def label_text_using_gpt(text, defn, name):
    chat_completion = client.chat.completions.create(
        messages=[
        {
            "role": "system", "content": "You are a helpful assistant for labelling text data."
        }, {
            "role": "user", "content": f"The following is part of a congressional speech. Given this definition of '{name}': '{defn}' and this text: '{text}', provide a score for the '{name}' of the text between -1 and 1. Please give me ONLY a number between -1 and 1 as your response."
         }
        
        ],
        model="gpt-3.5-turbo",
        )
    return chat_completion.choices[0].message.content


In [59]:
def safe_label_text_using_gpt_v2(texti, defn, name):
    while True: 
        try:
            return label_text_using_gpt(texti, defn, name)

        except openai.error.ServiceUnavailableError:  # Adjusted to be more specific
            print("\rServer is overloaded or not ready. Waiting for 5 minutes before retrying...         ", end="", flush=True)
            time.sleep(300)  # wait for 10 minutes before retrying

        except (openai.error.Timeout, requests.Timeout):  # Both timeout exceptions are now caught
            print("\rRequest timed out. Waiting for 5 minutes before retrying...        ", end="", flush=True)
            time.sleep(300)  # wait for 5 minutes before retrying

        except Exception as e:
            print(f"\rAn unexpected error occurred: {e}      ", end="", flush=True)
            return None

In [61]:
valence= "the degree of pleasure/positivity i.e. valence is the positive--negative or pleasure--displeasure dimension, where high valence (1) indicates the content of text is all positive or pleasurable and low valence (-1) indicates the content of the text is not positive or pleasurable" 
text_response_v= {} 
for i in range(0,len(df2)): 
    texti= df2.iloc[i].cleaned_text
    text_response_v[i] = safe_label_text_using_gpt_v2(texti, valence, "valence")
    if i%300 == 0: 
        print(i)
    

0
300
600
900
1200
1500
1800
2100
2400


In [62]:
valence_df= pd.DataFrame(text_response_v, index=[0]).T 
valence_df.to_csv("valence_df_115.csv", index=False)
df2["valence"] = list(valence_df[0])

In [63]:
arousal= "the degree of energy/excitedness i.e. arousal is the excited--calm or active--passive dimension, where high arousal (1) indicates the content of text is energetic and low arousal (-1) indicates the content of the text is not energetic and instead calm"
text_response_a= {} 
for i in range(0,len(df2)): 
    texti= df2.iloc[i].cleaned_text
    text_response_a[str(i)] = safe_label_text_using_gpt_v2(texti, arousal, "arousal")
    #time.sleep(.2) 
    if i%300 == 0: 
        print(i)

0
300
600
900
1200
1500
1800
2100
2400


In [64]:
arousal_df= pd.DataFrame(text_response_a, index=[0]).T 
arousal_df.to_csv("arousal_df_115.csv", index=False)
df2["arousal"] = list(arousal_df[0])

In [65]:
confidence= "the degree of confidence/assuredness i.e. confidence is the have full control--have no control dimension, where high confidence (1) indicates the content of text displays control and confidence and low confidence (-1) indicates the content of the text lacks control and confidence" 
text_response_c= {}
for i in range(0,len(df2)): 
    texti= df2.iloc[i].cleaned_text
    text_response_c[str(i)]= safe_label_text_using_gpt_v2(texti, confidence, "confidence")
    #time.sleep(.2) 
    if i%300 == 0: 
        print(i)



0
300
600
900
1200
1500
1800
2100
2400


In [66]:
confidence_df= pd.DataFrame(text_response_c, index=[0]).T 
confidence_df.to_csv("confidence_df_115.csv", index=False)
df2["confidence"] = list(confidence_df[0])

In [67]:
empathy= "the action of understanding, being aware of, being sensitive to, and vicariously experiencing the feelings, thoughts, and experience of another. i.e. Empathy involves stepping into that person's shoes to actively share in their emotional experience. Where high empathy (1) indicates the content of the text displays empathy and low empathy (-1) indicates the content of the text lacks empathy" 

text_response_e= {}
for i in range(0,len(df2)): 
    texti= df2.iloc[i].cleaned_text
    text_response_e[str(i)]= safe_label_text_using_gpt_v2(texti, empathy, "empathy")
    #time.sleep(.2) 
    if i%300 == 0: 
        print(i) 

0
300
600
900
1200
1500
1800
2100
2400


In [68]:
empathy_df= pd.DataFrame(text_response_e, index=[0]).T 
empathy_df.to_csv("empathy_df_115.csv", index=False)
df2["empathy"] = list(empathy_df[0])

In [69]:
sympathy= "an affinity, association, or relationship between persons or things wherein whatever affects one similarly affects the other, i.e. sympathy is a feeling of sincere concern for someone who is experiencing something difficult or painful. Where high sympathy (1) indicates the content of the text displays sympathy and low sympathy (-1) indicates the content of the text lacks sympathy" 

text_response_s= {}
for i in range(0,len(df2)): 
    texti= df2.iloc[i].cleaned_text
    text_response_s[str(i)]= safe_label_text_using_gpt_v2(texti, sympathy, "sympathy")
    #time.sleep(.2) 
    if i%300 == 0: 
        print(i)

0
300
600
900
1200
1500
1800
2100
2400


In [73]:
#sympathy_df= pd.DataFrame(text_response_s, index=[0]).T 
#sympathy_df.to_csv("sympathy_df.csv_115", index=False)
df2["sympathy"] = list(sympathy_df[0])

In [74]:
df2.to_csv("speeches_VACES_115.csv", index=False)